In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# For reproductibility
torch.manual_seed(1)

In [2]:
# Test softmax
z = torch.FloatTensor([1,2,3])
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [3]:
hypothesis.sum()

tensor(1.)

In [4]:
# Cross-entropy Loss (LOW-level)
z = torch.rand(3,5,requires_grad=True)
hypothesis = F.softmax(z,dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


In [5]:
y = torch.randint(5,(3,)).long()
print(y)

tensor([0, 2, 1])


In [9]:
# Cross-entropy Loss with torch.nn.functional (HIGH-level)
F.log_softmax(z, dim=1)
F.nll_loss(F.log_softmax(z,dim=1),y)
# Alternative Representation
F.cross_entropy(z,y)

tensor(1.4689, grad_fn=<NllLossBackward>)

In [14]:
x_train = [1,2,1,1, 2,1,3,2, 3,1,3,4, 4,1,5,5, 1,7,5,5, 1,2,5,6, 1,6,6,6, 1,7,7,7]
y_train = [2,2,2,1,1,1,0,0]
x_train = torch.FloatTensor(x_train).reshape(-1,4)
y_train = torch.LongTensor(y_train)
print(x_train)
print(y_train)

tensor([[1., 2., 1., 1.],
        [2., 1., 3., 2.],
        [3., 1., 3., 4.],
        [4., 1., 5., 5.],
        [1., 7., 5., 5.],
        [1., 2., 5., 6.],
        [1., 6., 6., 6.],
        [1., 7., 7., 7.]])
tensor([2, 2, 2, 1, 1, 1, 0, 0])


In [17]:
# Training with Low-level Cross-entropy Loss
# Training with F.cross_entropy
# Ininitalize Model
W = torch.zeros([4,3],requires_grad=True)
b = torch.zeros(1,requires_grad=True)
# Set Optimizer
optimizer = optim.SGD([W,b],lr = 0.01)

nb_epochs = 1000
for epoch in range(nb_epochs+1):
    
    # get Cost(1) - Low-level Cross Entropy Loss
#     hypothesis = F.softmax(x_train.matmul(W)+b ,dim=1)
#     y_one_hot = torch.zeros_like(hypothesis)
#     y_one_hot.scatter_(1,y_train.unsqueeze(1),1)
#     cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    # get Cost(2) - F.cross_entropy
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z,y_train)
    
    # Update H(x) from Cost
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # print logs each hundreds
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost : {:.6f}'.format(
            epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost : 1.098612
Epoch  100/1000 Cost : 0.850816
Epoch  200/1000 Cost : 0.784908
Epoch  300/1000 Cost : 0.744590
Epoch  400/1000 Cost : 0.714646
Epoch  500/1000 Cost : 0.690688
Epoch  600/1000 Cost : 0.670780
Epoch  700/1000 Cost : 0.653828
Epoch  800/1000 Cost : 0.639125
Epoch  900/1000 Cost : 0.626180
Epoch 1000/1000 Cost : 0.614640


In [19]:
# High-level Implementation with nn.Module
class SoftmaxClassierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4,3)
        
    def forward(self,x):
        return self.linear(x)
model = SoftmaxClassierModel()
optimizer = optim.SGD(model.parameters(),lr = 0.01)

nb_epochs = 1000
for epoch in range(nb_epochs+1):
    
    # get Cost(2) - F.cross_entropy
    prediction = model(x_train)
    cost = F.cross_entropy(prediction,y_train)
    
    # Update H(x) from Cost
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # print logs each hundreds
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost : {:.6f}'.format(
            epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost : 1.849513
Epoch  100/1000 Cost : 0.839934
Epoch  200/1000 Cost : 0.757753
Epoch  300/1000 Cost : 0.704903
Epoch  400/1000 Cost : 0.665221
Epoch  500/1000 Cost : 0.633748
Epoch  600/1000 Cost : 0.608009
Epoch  700/1000 Cost : 0.586476
Epoch  800/1000 Cost : 0.568120
Epoch  900/1000 Cost : 0.552216
Epoch 1000/1000 Cost : 0.538244
